# Exam details
Written work:  

You will have to analyze a dataset and test a relevant hypothesis.

You should choose your own dataset by Lecture 12 (written hand-in already available)

The report will be written as a Jupiter notebook.

Minimal set of analysis:
1) Describe your data and visualize some key dimensions.
2) Perform at least two analysis (depending on what is appropriate given the data you selected):
- Does your data contain quantitative values that allow for a hypothesis testing?
   IF YES: Formulate an hypothesis and test it. Complement the testing with an appropriate visualization.
- Does your data contain unstructured textual information?
 IF YES: Perform sentiment analysis on your data and describe and visualize the results.
- Does your data contain network structures (or a network structure can be extracted)?
 IF YES: Ask a question about the network structure and answer it.

OBS: Different datasets can be investigated in many different ways. Any combination of the above-described analysis is acceptable as long as you ask 2 questions. 
e.g. Statistical hypothesis testing + network analysis, Sentiment analysis + network analysis , Statistical hypothesis testing + Statistical hypothesis testing.

Groups:  Find your own group (2 ppls - 3 is possible) let the TAs know before Fall break.

Where to find the data:
The internet is full of data, these are just few starting points:
Data (digst.dk) - Danish Open Data
GHO | By theme (who.int) - WHO Open Data
DataHub  - Many datasets
Data.gov Home - Data.gov - US Open Data
Kaggle - All sort of datasets
Dataset Search (google.com)
TLC Trip Record Data - TLC (nyc.gov) - TAXI data in NYC
Industry data and insights | BFI - Movies!
CDE (cjis.gov) - Crime data from FBI
Københavns Kommune (opendata.dk) - Open Data from CPH

Opgaven skal indeholde en eller flere af følgende: 
- Hypothesis testing
- Sentiment Analyse
- Network Analysis 

Desuden også
- Interactive Visualization 

# Data
Datasættet stammer fra "The Danish Parliament Corpus 2009 - 2017, v2, w. subject annotation".

Kilde: 
Hansen, Dorte Haltrup and Navarretta, Costanza, 2021, The Danish Parliament Corpus 2009 - 2017, v2, w. subject annotation, CLARIN-DK-UCPH Centre Repository, http://hdl.handle.net/20.500.12115/44.


Datasættet består af transkriptioner af taler i Folketinget fra første samling 2009 til og med først samling 2016 (6/10 2009 – 7/9 2017). Til hver tale er der tilknyttet metadata, dels om medlemmet af folketinget ('Name', 'Gender', 'Party', 'Role', 'Title', 'Birth', 'Age'), dels om talen (Date', 'samling', 'Start time', 'End time', 'Time', 'Agenda item', 'Case no', 'Case type', 'Agenda title', 'Subject 1', 'Subject 2').

Datasættet er struktureret i tsv txt-filer, som er formateret i utf-8. Der er en fil per møde.

Kilde:
Samme, Readme


Til denne opgave har vi samlet tsv filerne i et nyt datasæt, som vi har gemt i en csv fil separeret med pipes. Csv filen er uploadet til sciencedata.dk, hvorfra den kan downloades via url med pandas.read_csv() metoden.





# Emnet
Emnet er immigrationspolitik fra 2009 - 2017. Hvilke kendetegn har de forskellige partiers politik vurdereret ud fra partimedlemers taler i Folketinget?

1. Ved hjælp af Tf-Idf identificerer vi de særegne nøgleord, der kendetegner de forskellige partier.

2. sentiment analyse på taler som indeholder noget om flygtning. Det kan f.eks. være taler, som handler om 'os' og 'dem' - nærlæsning. Bliver der større variation i sentiment-scorrerne mere varieret op til et valg? Bliver de mere varieret omkring 2015?  
3. pos -tag f.eks. verber fra forskellige partier, og hvilke adjektiver knytters sig til et begreb. 


# Load data

Data ligger på sciencedata.dk og deles derfra med download link.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('https://sciencedata.dk/shared/825e999a5c13fd22d28d4289fa899ba1?download', sep='|')

In [ ]:
print (f'The coloumns are {df.columns}')

In [ ]:
#df

# Question 1
# TF-IDF. Segment = speech 

1. Ved hjælp af Tf-Idf identificerer vi de særegne nøgleord, der kendetegner de forskellige partier.

Method:
1. Subset data on the subject value "Immigration" and on the role value "member". Group on session and party and aggregate the speeches according to the groups.  
2. Preprocess the texts using a stopword list and ***Spacy*** to lemmatize words in speeches.
3. Use **Tf-Idf** to identify distinctive keywords



Der skal renses bedre, fordi spacy lemmatizer løber tør for plads med så mange ord.
Jeg kan fjerne ord med tal, tal og ord mindre end to bogstaver.


ValueError: [E088] Text of length 1975499 exceeds maximum of 1000000. The parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.


In [ ]:
# Subset data
input_data = df[(df['Subject 1'] == 'Immigration') & (df['Role'] == 'medlem')].reset_index()
# Group by 'session' and 'party' and aggregate speeches
input_data_grouped = df.groupby(['samling', 'Party'])['Text'].agg(' '.join).reset_index()

In [ ]:
# Preprocess
import re

def scrub_text(text):
    return re.findall(r'\b\S+\b', text.lower().replace('_', ' '))

# load stopword list
with open('dk.txt', 'r', encoding='utf-8-sig') as f:
    stop_words = f.read().split('\n')    
    
def filter_stopword(text_list):
    return [i for i in text_list if i not in stop_words]
  
print('speeches')    
speeches = input_data_grouped['Text'].tolist()

print('clean_strings_in_list')
clean_strings_in_list = [scrub_text(i) for i in speeches]

print('strings_wo_stop_words')
strings_wo_stop_words = [filter_stopword(text) for text in clean_strings_in_list] 

print('strings')
strings = [' '.join(i) for i in strings_wo_stop_words]

In [ ]:
input_data_grouped['Clean_text_wo_sw'] = strings

Hvis jeg kører lemmatizer koden nedenfor får jeg:

_ValueError: [E088] Text of length 1975499 exceeds maximum of 1000000. The parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`._

Derfor skal jeg koorigere planen, opdele data i mindre subsets og køre funktionen på mindre dele.

In [ ]:
input_data_grouped

In [ ]:
import spacy

# Load Danish spacy model
nlp = spacy.load("da_core_news_sm")


def lemmatize_text(text):
    # Process the text with spaCy
    doc = nlp(text)

    # Extract the lemmatized tokens and join them back into a string
    lemmatized_text = " ".join([token.lemma_ for token in doc])

    return lemmatized_text


In [ ]:
input_div_1 = input_data_grouped.iloc[0:20, [0, 1, 3]]
input_div_2 = input_data_grouped.iloc[20:40, [0, 1, 3]]
input_div_3 = input_data_grouped.iloc[40:60, [0, 1, 3]]
input_div_4 = input_data_grouped.iloc[60:80, [0, 1, 3]]
input_div_5 = input_data_grouped.iloc[80:100, [0, 1, 3]]
input_div_6 = input_data_grouped.iloc[100:120, [0, 1, 3]]

In [ ]:
input_div_1

In [ ]:
input_div_1['Lemmatized_text'] = input_div_1['Clean_text_wo_sw'].apply(lambda x : lemmatize_text(x))

In [ ]:
len(input_div_1.at[0,'Clean_text_wo_sw'])

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer

# load stopword list
with open('dk.txt', 'r', encoding='utf-8-sig') as f:
    stop_words = f.read().split('\n')

def top_distinctive_words(documents):
    # Create a TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer(max_features=10000)

    # Fit and transform the input documents
    tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

    # Get the feature names (words)
    feature_names = tfidf_vectorizer.get_feature_names_out()

    # Initialize a list to store the top distinctive words for each document
    top_words_list = []

    # Iterate through the TF-IDF matrices for each document
    for tfidf_vector in tfidf_matrix.toarray():
        # Create a list of tuples (word, TF-IDF score) for the current document
        word_tfidf_tuples = [(feature, tfidf_score) for feature, tfidf_score in zip(feature_names, tfidf_vector) if tfidf_score > 0]

        # Sort the tuples by TF-IDF score in descending order
        word_tfidf_tuples.sort(key=lambda x: x[1], reverse=True)

        # Select the top five words with the highest TF-IDF scores
        top_words = [word for word, _ in word_tfidf_tuples[:20]]

        top_words_list.append(top_words)

    return top_words_list



documents = input_data_grouped['Text'].tolist()


top_words_list = top_distinctive_words(documents)

input_data_grouped['distinctive_keywords'] =  top_words_list



# Question 3

Sentiment analyse af taler som indeholder noget om flygtning. Det kan f.eks. være taler, som handler om 'os' og 'dem' - nærlæsning. Bliver der større variation i sentiment-scorerne mere varieret op til et valg? Bliver de mere varieret omkring 2015?

M: Brug sentiment analyse til at identificere forandringer i sentimenter i taler knyttet til emneordet Immigration.
Definition: "SA is a part of applied computational linguistics and attempts to quantify the emotions" (Kran, E., & Orm, S. (2020)).


### Hvilken sentiment analyse tilgang anvender vi?

På Alexandra Instituttets DaNLP repository, et repository for Natural Language Processing resources for the Danish Language, leverer Alexandra Instituttet et overblik over open sentiment analysis models and dataset for Danish.

Der er følgende modeller:
1. AFINN - wordlist model, der returnerer en tal-score, der modsvarer en følelse i et ord. Negativ ( minus ), neutral (0) eller positiv ( plus ) 
2. Sentida - wordlist model, der ligesom ovenfor returnerer en tal-score, der modsvarer en følelse.
3. BERT Emotion - BERT model, der returner en beskrivende tekststreng, der modsvarer et semantisk felt, som et ord er indlejret i, f.eks. glæde/sindsro, forventning/interesse, tillid/accept.     
4. BERT Tone - Bert model, der ligeledes returner en beskrivende tekststreng.
5. SpaCy Sentiment - Spacy model, der også returner en tekststreng.
6. Senda - Bert model, der også returner en tekststreng. 

De to første, AFINN og Sentida, samt den sidste Senda, er ikke en del af danlp projektet og dermed ikke en del af DaNLP Python projektet, som kan anvendes via pip. 

Kilde: Alexandra Institute. (2021). Sentiment_analysis.md. https://github.com/alexandrainst/danlp/blob/master/docs/docs/tasks/sentiment_analysis.md


I denne opgave vil vi anvende sentiment analyse til at se på variationer i scorerne. Vi kan derfor vælge en af de første to, hvorved vi også vælger ikke at anvende DaNLPs Python pakke. Af de to første wordlist modeller, AFINN og Sentida vælger vi Sentida, fordi modellen er blevet opdateret med ny funktionalitet, og fordi set ud fra Finn Årup Nielsens publicationer siden 2019 har han ikke udgivet nyt omkring sentiment analyse i den periode. 

Både AFINN og Sentida er wordlist modeller, der aggregerer en sentiment scores baseret på forekomsten af ord fra ordlisten i en given tekst. Den aggregerede score bliver anvendt som en indikator på, hvor positiv teksten 
er. Der er fire problemer forbundet med denne model:
1. modellen tager ikke højde for syntaktiske relation mellem ord.
2. modellen ignorerer adverbier, og dermed den betydning adverbierne har i at udtykke grader af noget og holdinger til noget.
3. modellen afspejler ikke menneskers måde at opfatte følelser.
4. modellen kan ikke håndtere ord der betyder to forskellige ting.
Udviklerne bag Sentida har fundet inspiration i den engelsk sprogede VADER model og har forsøgt at minimere problemerne ved at indbygge forskellige, simple former for "awareness". For eksempel at forøge score omkring negationer, captital letter og udråbstegn for at give disse elementer i sproget større opmærksomhed. 

For example:

_“Maden (+0.3) var god (+2.3), (← x 0.5) men(1.5 x →) serviceringen (+0.3) var elendig (-4.3).” ⇒ 1.3 -6 ⇒ sentiment score: -4.7 

“The  food (+0.3)was  good (+2.3),  (←  x  0.5) but(1.5x  →) the  service (+0.3)was horrendous (-4.3).” ⇒ 1.3 -6 ⇒ sentiment score: -4.7_


_“Det er så sejt (+3.6)!(← x 1.291)” ⇒ sentiment score: 4.6

“It is so cool (+3.6)!(←x 1.291)”⇒ sentiment score: 4.6_


_“DET ERSÅ SEJT (+3.6). (← x 1.733)” ⇒ sentiment score: 6.2

“IT IS SO COOL (+3.6). (← x 1.733)” ⇒ sentiment score: 6.2_


(Kran, E., & Orm, S. (2020)).






Neural networks baserede SA modeller som "aspect-based sentiment analysis" kan tage højde for ord og kontekst og er på den måde mere præcise, men eftersom neurale networds modeller forudsætter store mængder af træningsdata, er de langsomme og upraktiske at benytte (Kran, E., & Orm, S. (2020)).

In [ ]:
# pip install sentida

In [ ]:
from sentida import Sentida
# Define the class:
SV = Sentida()

SV.sentida(
        text = 'Lad der blive fred.',
        output = 'mean',
        normal = False,
        speed = 'normal')

https://github.com/alexandrainst/danlp/blob/master/docs/docs/tasks/sentiment_analysis.md

https://github.com/alexandrainst/danlp/blob/master/docs/docs/frameworks/spacy.md

# Litteraturliste

Lauridsen, G. A., Dalsgaard, J. A., & Svendsen, L. K. B. (2019). SENTIDA: A New Tool for Sentiment Analysis in Danish. Journal of Language Works - Sprogvidenskabeligt Studentertidsskrift, 4(1), 38–53. Retrieved from https://tidsskrift.dk/lwo/article/view/115711

Kran, E., & Orm, S. (2020). EMMA: Danish Natural-Language Processing of Emotion in Text: The new State-of-the-Art in Danish Sentiment Analysis and a Multidimensional Emotional Sentiment Validation Dataset. Journal of Language Works - Sprogvidenskabeligt Studentertidsskrift, 5(1), 92–110. Retrieved from https://tidsskrift.dk/lwo/article/view/121221

Brogaard Pauli, Amalie and Barrett, Maria and Lacroix, Ophélie and Hvingelby, Rasmus. (2021) An open-source toolkit for Danish Natural Language Processing. https://ep.liu.se/ecp/178/053/ecp2021178053.pdf 

Alexandra Institute. (2021). Sentiment_analysis.md. https://github.com/alexandrainst/danlp/blob/master/docs/docs/tasks/sentiment_analysis.md

Hansen, Dorte Haltrup and Navarretta, Costanza, 2021, The Danish Parliament Corpus 2009 - 2017, v2, w. subject annotation, CLARIN-DK-UCPH Centre Repository, http://hdl.handle.net/20.500.12115/44.

McKinney, Wes. Python for Data Analysis : Data Wrangling with Pandas, NumPy and Jupyter. Third edition. Sebastopol, CA: O’Reilly Media, Inc., 2022

In [ ]:
# bump_chart
# https://altair-viz.github.io/gallery/bump_chart.html